In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U -q transformers
import sys
sys.path.append('/content/drive/MyDrive/02.git/healthcare-hackathon/model/fer2013-finetuning')

from PIL import Image
import numpy as np
import torch
from models.classifier import create_emotion_classifier
from utils.dataset import create_dataloaders
from utils.trainer import EmotionTrainer
from configs.config import Config

# 모델 생성
model = create_emotion_classifier(
    num_classes=Config.NUM_CLASSES,
    backbone_name=Config.MODEL_NAME
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/39.2M [00:00<?, ?B/s]

In [32]:
path_ = '/content/drive/MyDrive/05.share/'

epoch_m, epoch_v = '10', '9'

model_name_mobile = f'model-weights/mobilenet_epoch_{epoch_m}.pth'
model_name_vit = f'model-weights/ViT_epoch_{epoch_v}.pth'

path_vit = path_+model_name_vit

path_mobile = path_+model_name_mobile
path_mobile

'/content/drive/MyDrive/05.share/model-weights/mobilenet_epoch_10.pth'

In [22]:
# 저장된 가중치 로드
checkpoint = torch.load(path_mobile, map_location=('cuda' if torch.cuda.is_available() else 'cpu'))
model.load_state_dict(checkpoint)
print("✅ best_model_copy.pth 로드 완료")

# 모델의 디바이스 확인
model.to(Config.DEVICE)
device = next(model.parameters()).device
print(f"모델은 현재 {device}에 있습니다.")

✅ best_model_copy.pth 로드 완료
모델은 현재 cuda:0에 있습니다.


In [12]:
# 데이터셋 압축해제
!unzip -q {path_}fer2013.zip -d /content/fer2013

replace /content/fer2013/test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm


# 테스트 데이터셋 로드
train_loader, val_loader = create_dataloaders(
    data_dir='/content/fer2013',
    batch_size=256,
    num_workers=2
)

# 모델을 평가 모드로 전환
model.eval()

# 예측 및 실제 레이블 저장
all_preds = []
all_labels = []

# 테스트 데이터셋에 대해 예측 수행
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):  # tqdm으로 반복문 감싸기
        inputs, labels = inputs.to(Config.DEVICE), labels.to(Config.DEVICE)

        # 모델 예측
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # 예측 및 실제 레이블 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)
print(f"테스트 정확도: {accuracy * 100:.2f}%")

# 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
class_labels = ['angry', 'disgust', 'fear',
            'happy', 'neutral', 'sad', 'surprise']

# 분류 보고서 출력
report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_labels])
print(report)

Evaluating: 100%|██████████| 29/29 [00:15<00:00,  1.93it/s]

테스트 정확도: 34.15%
                precision    recall  f1-score   support

   Class angry       0.20      0.59      0.30       958
 Class disgust       0.00      0.00      0.00       111
    Class fear       0.43      0.00      0.01      1024
   Class happy       0.69      0.62      0.66      1774
 Class neutral       0.54      0.08      0.13      1233
     Class sad       0.26      0.55      0.36      1247
Class surprise       0.44      0.00      0.01       831

      accuracy                           0.34      7178
     macro avg       0.37      0.26      0.21      7178
  weighted avg       0.45      0.34      0.29      7178




/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
def huggingface_model_evaluation(model, processor, val_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    # 예측 및 실제 레이블 저장
    all_preds = []
    all_labels = []

    # 테스트 데이터셋에 대해 예측 수행
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Evaluating"):
            # [0,1] 텐서를 PIL Image로 변환
            batch_images = []
            for img_tensor in inputs:
                # CHW → HWC 순서 변경 후 [0,1] → [0,255] 변환
                img_array = img_tensor.permute(1, 2, 0).numpy()
                img_array = (img_array * 255).astype(np.uint8)
                batch_images.append(Image.fromarray(img_array))

            # Hugging Face processor가 자동으로 모델에 맞는 전처리 수행
            processed_inputs = processor(images=batch_images, return_tensors="pt").to(Config.DEVICE)

            outputs = model(**processed_inputs)
            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 정확도 계산
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"테스트 정확도: {accuracy * 100:.2f}%")

    # 클래스 레이블 (예: 0부터 6까지의 7가지 클래스)
    class_names = ['angry', 'disgust', 'fear',
                'happy', 'neutral', 'sad', 'surprise']

    # 분류 보고서 출력
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in class_names])
    print(report)

In [33]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("mo-thecreator/vit-Facial-Expression-Recognition", use_fast=True)
model_mo = AutoModelForImageClassification.from_pretrained("mo-thecreator/vit-Facial-Expression-Recognition")

# 저장된 가중치 로드
model_mo.load_state_dict(torch.load(path_vit, map_location = ('cuda' if torch.cuda.is_available() else 'cpu')))

model_mo.to(Config.DEVICE)
huggingface_model_evaluation(model_mo, processor, val_loader)

Evaluating: 100%|██████████| 29/29 [01:40<00:00,  3.46s/it]

테스트 정확도: 16.23%
                precision    recall  f1-score   support

   Class angry       0.14      0.86      0.24       958
 Class disgust       0.00      0.00      0.00       111
    Class fear       0.15      0.00      0.01      1024
   Class happy       0.58      0.02      0.04      1774
 Class neutral       0.21      0.09      0.13      1233
     Class sad       0.26      0.15      0.19      1247
Class surprise       0.17      0.00      0.00       831

      accuracy                           0.16      7178
     macro avg       0.22      0.16      0.09      7178
  weighted avg       0.28      0.16      0.10      7178




/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
def explain():
  """
  mob 3 -> 테스트 정확도: 33.88%
  ViT 4 -> 테스트 정확도: 17.40%

  mob 6 -> 테스트 정확도: 33.42%
  ViT 9 -> 테스트 정확도: 16.23%

  mob 10 -> 테스트 정확도: 34.15%
  ViT 10 -> 테스트 정확도: 16.23%
  """